# Association Rule Learning

在数据挖掘与机器学习中，关联规则（Association Rules）是一种较为常用的无监督学习算法，与我们前面所学习的分类、聚类等算法的不同的是，这一类算法的主要目的在于——发掘数据内在结构特征（即变量）之间的关联性，**暗示两种物品之间可能存在很强的关系**。

简单一点来说，就是在大规模的数据集中寻找一些有意义有价值的关系。有了这些关系，一方面，可以帮助我们拓宽对数据及其特征的理解；另一方面，则可以实现推荐系统的构建与应用（例如购物篮分析等）。

在对关联规则有了基本的认识后，我们对其进行进一步的细分，以日常生活中的关联性举例，在逛超市的顾客中，购买面包的人很大程度上会购买牛奶，这一类的关联性被称为<b>简单关联规则</b>；再例如，购买汽车遮阳板的很多顾客会在近期内购买零度玻璃水，这样的事例不仅反映了事物间的关联关系， 而且还具有时间上的先后顺序，因此这一类的关联性则被称为<b>序列关联规则</b>。

广义上的关联规则包含了简单关联和序列关联

## Apriori (先验算法)

[一步步教你轻松学关联规则Apriori算法](https://bainingchao.github.io/2018/09/27/%E4%B8%80%E6%AD%A5%E6%AD%A5%E6%95%99%E4%BD%A0%E8%BD%BB%E6%9D%BE%E5%AD%A6%E5%85%B3%E8%81%94%E8%A7%84%E5%88%99Apriori%E7%AE%97%E6%B3%95/)<br>
[Apriori 算法简介及 python3实现](https://zhuanlan.zhihu.com/p/39918644)

实际上，在数据中使用关联分析进行探索时，我们可以找出很多关联规则，但并非所有的关联规则都是有效的，有的可能令人信服的程度并不高，也有的可能适用范围很有限，带有这些特征的所谓的“关联规则”，我们则称之为不具有<b>“有效性”</b>。

判断一条关联规则是否有效，需要用到以下两大测度指标，即<b>规则支持度</b>与<b>规则置信度</b>。

**规则支持度 (Support)**<br>
支持度测量了简单关联规则应用的普适性，**定义为项目$A$与项目$B$同时出现的概率**，数学表述为：$Support(A \rightarrow B) = P(A \cap B)$<br>
假设某天共有100个顾客到商场购买物品，其中有10个顾客同时购买了电脑和杀毒软件，那么上述关联规则的支持度就为10％。同样，支持度越高，表明某一关联规则的适用性越大

**规则置信度 (Confidence)**<br>
置信度是对简单关联规则准确度的测量，**定义为包含项目$A$的事务中同时也包含项目$B$的概率**，数学表述为：$Confidence(A \rightarrow B) = P(B|A) = \frac{P(A \cap B)}{P(A)}$<br>
置信度的本质就是我们所学过的条件概率，置信度越高，则说明$A$出现则$B$出现的可能性也就越高。假设在电脑$\rightarrow$杀毒软件的关联规则中，置信度$C = 60$%，表示购买电脑的顾客中有$60$%的顾客也购买了杀毒软件。

一个有效的简单关联规则，势必同时具有较高的置信度与支持度。因为，如果支持度较高而置信度较低，则证明规则的可信度差；而相反，如果支持度较低而置信度较高，则说明规则的应用范围较小。

举例来讲，假设在1000个顾客购买行为的事务中，只有1个顾客购买了烧烤炉，同时也只有他购买了碳，虽然规则“烧烤炉$\rightarrow$碳”的置信度很高，为$100$%，但其支持度仅有$0.1$%，说明这条规则缺乏普遍性，应用价值不高。

所以，一个有效的关联规则，必须具有较高的置信度与支持度。那么在实际应用中，我们就需要给定最小的置信度$C_{min}$与支持度$S_{min}$，只有同时大于$C_{min}$和$S_{min}$的规则，我们才可以将其定义为是“**有效**”的。

如何衡量关联规则具有实用性呢？这里我们就需要借助规则的提升度了。

**规则提升度 (Lift)** 定义为：置信度与后项支持度之比，数学表述为：<br>
$Lift(A \rightarrow B) = \frac{Confidence(A \rightarrow B)}{P(B)} = \frac{P(A \cap B)}{P(A)P(B)}$

提升度反映了项目$A$的出现对项目$B$出现的影响程度。从统计角度来看，如果$A$的出现对项$B$的出现没有影响，即$A$与$B$相互独立的话，$P(A ∩ B) = P(A)P(B)$，此时规则提升度为1。所以，具有实用性的关联规则应该是提升度大于1的规则，即$A$的出现对$B$的出现有促进作用。同样，提升度越大，证明规则实用性越强。

### Apriori 原理

假设我们一共有 4 个商品: 商品0, 商品1, 商品2, 商品3。 所有可能的情况如下:
<img src='https://github.com/yunjcai/Machine-Learning-A-Z/blob/master/Part%205%20-%20Association%20Rule%20Learning/apriori_1.JPG?raw=true' width='300'>
如果我们计算所有组合的支持度，也需要计算 $15$ 次。即 $2^N−1=2^4−1=15$。随着物品的增加，计算的次数呈指数的形式增长 。为了降低计算次数和时间，研究人员发现了一种所谓的 Apriori 原理，即某个项集是频繁的，那么它的所有子集也是频繁的。 例如，如果 {$0$, $1$} 是频繁的，那么 {$0$}, {$1$} 也是频繁的。 该原理直观上没有什么帮助，但是如果反过来看就有用了，也就是说如果一个项集是 非频繁项集，那么它的所有超集也是非频繁项集，如下图所示:
<img src='https://github.com/yunjcai/Machine-Learning-A-Z/blob/master/Part%205%20-%20Association%20Rule%20Learning/apriori_2.JPG?raw=true' width='300'>
在图中我们可以看到，已知灰色部分 {$2$,$3$} 是 非频繁项集，那么利用上面的知识，我们就可以知道 {$0,2,3$} {$1,2,3$} {$0,1,2,3$} 都是 非频繁的。 也就是说，计算出 {$2,3$} 的支持度，知道它是 非频繁 的之后，就不需要再计算 {$0,2,3$} {$1,2,3$} {$0,1,2,3$} 的支持度，因为我们知道这些集合不会满足我们的要求。 使用该原理就可以避免项集数目的指数增长，从而在合理的时间内计算出频繁项集。

### Apriori Algorithm

**STEP 1:** Set a minimum $Support$ and $Confidence$ (设置最小的 limitation 来限制大量的排列组合。如排除 support 低于$20$% 的物品)<br>
**STEP 2:** Take all the subsets in $A$ having higher $Support$ than minimum $Support$<br>
**STEP 3:** Take all the rules of these subsets having higher $Confidence$ than minimum $Confidence$<br>
**STEP 4:** Sort the rules by decreasing $Lift$ (The highest $Lift$ are the ones we need consider for actually implementing business decision)

**要做的工作：**<br>
1. 生成频繁项集：这一阶段找出所有满足最小支持度的项集（具有统计学意义的组合），找出的这些项集称为频繁项集。自信度与支持度的计算涉及到多个列表的循环，极大影响频繁项集的生成时间。<br><br>
2. 生成关联规则：在上一步产生的频繁项集的基础上生成满足最小自信度的规则，产生的规则称为强规则。

**两个重要的定理：**<br>

1. 如果一个集合是频繁项集，则它的所有子集都是频繁项集。假设一个集合{A,B}是频繁项集，则它的子集{A}, {B} 都是频繁项集。<br><br>
2. 如果一个集合不是频繁项集，则它的所有超集都不是频繁项集。假设集合{A}不是频繁项集，则它的任何超集如{A,B}，{A,B,C}必定也不是频繁项集。<br>
<img src='https://github.com/yunjcai/Machine-Learning-A-Z/blob/master/Part%205%20-%20Association%20Rule%20Learning/apriori_4.JPG?raw=true'>

### 实例理解 1

假设有一个数据库D，其中有4个事务记录，分别表示为：<br>

|TID|Items|
|---|-----|
|T1|l1,l3,l4|
|T2|l2,l3,l5|
|T3|l1,l2,l3,l5|
|T4|l2,l5|

这里预定最小支持度 $minSupport = 2$,下面用图例说明算法运行的过程：<br>
1. 扫描D，对每个候选项进行支持度计数得到表C1:

|项集|支持度计数|
|---|---|
|{l1}|2|
|{l2}|3|
|{l3}|3|
|{l4}|l|
|{l5}|3|

2. 比较候选项支持度计数与最小支持度 $minSupport$（假设为2），删除低于最小支持度的，产生1维最大项目集L1：

|项集|支持度计数|
|---|---|
|{l1}|2|
|{l2}	|3|
|{l3}|	3|
|{l5}|	3|

3. 两两组合后，由L1产生候选项集C2，根据数据库D中信息(表1)对每个候选项集进行支持度计数：

|项集|	支持度计数|
|---|---|
|{l1,l2}|	1|
|{l1,l3}	|2|
|{l1,l5}	|1|
|{l2,l3}	|2|
|{l2,l5}	|3|
|{l3,l5}	|2|

4. 比较候选项支持度计数与最小支持度 $minSupport$，删除低于最小支持度的，产生2维最大项目集L2：

|项集|	支持度计数|
|---|---|
|{l1,l3}	|2|
|{l2,l3}	|2|
|{l2,l5}	|3|
|{l3,l5}	|2|

5. 由L2产生候选项集C3：

|项集|支持度计数|
|---|---|
|『l1,l3,l5}|1|
|{l2,l3,l5}|2|

6. 比较候选项支持度计数与最小支持度$minSupport$，删除低于最小支持度的，产生3维最大项目集L3：

|项集	|支持度计数|
|---|---|
|{l2,l3,l5}	|2|

### 实例理解 2

首先收集所有数据集（可以理解为商品清单），经过数据预处理后如Database TDB所示。
<img src='https://github.com/yunjcai/Machine-Learning-A-Z/blob/master/Part%205%20-%20Association%20Rule%20Learning/apriori_3.JPG?raw=true' width='400'>

<b>第一步</b>对每个候选项进行支持度计数得到表C1，比较候选项支持度计数与最小支持度 $minSupport$（假设最小支持度为2），产生1维最大项目集L1。再对L1进行组合产生候选项集C2。<br>
<b>第二步</b>我们对C2进行支持度计数，比较候选项支持度计数与最小支持度 $minSupport$，产生2维最大项目集L2。由L2产生候选项集C3。<br>
<b>第三步</b>对C3进行支持度计数，使用Apriori性质剪枝：频繁项集的所有子集必须是频繁的，对候选项C3，我们可以删除其子集为非频繁的选项，{A,B,C}的2元素子集是{A,B},{A,C},{B,C}，其中{A,B}不是L2的元素，所以删除这个选项；{A,C,E}的2元素子集是{A,C},{A,E},{C,E}，其中{A,E} 不是L2的元素，所以删除这个选项；{B,C,E}的2元素子集是{B,C},{B,E},{C,E}，它的所有2－项子集都是L2的元素，因此保留这个选项。这样，剪枝后得到{B,C,E}，比较候选项支持度计数与最小支持度 $minSupport$，产生3维最大项目集L3：继续进行没有满足条件，算法终止。

### Apriori in Python

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Making Apriori model for a store, so we can find out the association rules of the different products to see how the manager can optimize the placement of its different products to optimize the sales. 

In [2]:
# data第一行并不是 titles for the columns，而是表格内容。
dataset = pd.read_csv('Market_Basket_Optimisation.csv',header=None)
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The Apriori is expecting the input is actually a list that contains the different transactions. Each transaction puts in a list<br>
So there'll be a one big list containing all the different transcations and each transcation is going to be a list itself.

In [3]:
transactions = []

for i in range(0,dataset.shape[0]):
    # need each transaction to be a list rather than whole one list and cast it to be string
    # each row in dataset becomes a string list
    transactions.append([str(dataset.values[i,j]) for j in range(0,dataset.shape[1])])

In real life, Try different values of Support and Confidence until we are satisfied with the rules and we think it makes sense. We can also try these rules within a certain period of time, and then if we look at the impact on the revenue and if we don't observe a meaningful increase in the sales revenue we can later change the Support and the Confidence to change the rules until we find the strongest rules that optimize the sales.

In [4]:
from apyori import apriori
# min_length allows us to set the minimum number of products we want to have in our rules
# Need at least 2 products in the baskets in the transactions
# In this example, we consider the products are purchased 3 times a day 
# min_support = 3x7 (week) /total transactions = 21/7500 = 0.003
# min_confidence : the default value was 0.8, it means the rules has to be correct in 80% of the time.
# 80% is too high that we cannot get any rule because there was no rule being correct 80% of the time.
# so we try 80%/2 first, then to see whether the rule would make sense (will have water & egg with high releavent)
# then we try 40%/2 again, we decide 20% is a good choice.
# The Lift is a great insight of the releavance and the strength of rule, so let's use 3.
rules = apriori(transactions,min_support=0.003,min_confidence=0.2,min_lift=3,min_length=2)

The 'rules' has already been sorted by their relevance in Apriori model. It's actually a combination of the Support, the Confidence and the Lift.<br>
Each row is the top releavant of rule

In [5]:
results = list(rules)
print(results)

[RelationRecord(items=frozenset({'light cream', 'chicken'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)]), RelationRecord(items=frozenset({'mushroom cream sauce', 'escalope'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), confidence=0.3006993006993007, lift=3.790832696715049)]), RelationRecord(items=frozenset({'pasta', 'escalope'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pasta'}), items_add=frozenset({'escalope'}), confidence=0.3728813559322034, lift=4.700811850163794)]), RelationRecord(items=frozenset({'honey', 'fromage blanc'}), support=0.003332888948140248, ordered_statistics=[OrderedStatistic(items_base=frozenset({'fromage blanc'}), items_add=frozenset({'honey'}), confidence=0.24